In [1]:
!pwd

/Users/sumitmishra/Desktop/GitHub/Credit-Default-Risk-Prediction


In [2]:
import os
import pandas as pd
pd.options.display.max_columns = None  # Show all columns
pd.options.display.width = 1000        # Adjust width to fit more data

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

spark = SparkSession.builder.appName("Home Credit Default Prediction").getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True) 
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)
spark.conf.set("spark.sql.repl.eagerEval.maxNumCols", 200) 
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

path = os.getcwd()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/07 10:07:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
sdf_train = spark.read.option('header', True)\
           .csv(path + "/Data/application_train.csv")

sdf_bureau = spark.read.option('header', True)\
           .csv(path + "/Data/bureau.csv")

sdf_bureau_balance = spark.read.option('header', True)\
           .csv(path + "/Data/bureau_balance.csv")

In [7]:
sdf_train.show(10, False)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+-----------------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+----------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-

In [9]:
sdf_bureau.show(10, False)

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|215354    |5714462     |Closed       |currency 1     |-497       |0                 |-153.0             |-153.0         

In [8]:
sdf_bureau_balance.show(10, False)

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|5715448     |0             |C     |
|5715448     |-1            |C     |
|5715448     |-2            |C     |
|5715448     |-3            |C     |
|5715448     |-4            |C     |
|5715448     |-5            |C     |
|5715448     |-6            |C     |
|5715448     |-7            |C     |
|5715448     |-8            |C     |
|5715448     |-9            |0     |
+------------+--------------+------+
only showing top 10 rows



In [47]:
sdf_train.filter(F.col('SK_ID_CURR') == 215354)\
            .sort(F.asc('SK_ID_CURR'))\
            .show(20, False)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-------------------+------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-----------------+------------------+-----------------+------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----

In [33]:
sdf_bureau.filter(F.col('SK_ID_CURR') == 215354)\
            .sort(F.asc('SK_ID_BUREAU'))\
            .show(20, False)

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|215354    |5714458     |Closed       |currency 1     |-1872      |0                 |-776.0             |-1203.0        

In [69]:
sdf_bureau.filter(F.col('SK_ID_CURR') == 215354).select('AMT_ANNUITY', 'SK_ID_CURR').show(20, False)

+-----------+----------+
|AMT_ANNUITY|SK_ID_CURR|
+-----------+----------+
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
+-----------+----------+



In [67]:
sdf_train.filter(F.col('SK_ID_CURR') == 215354).select('AMT_ANNUITY', 'SK_ID_CURR').show(20, False)

+-----------+----------+
|AMT_ANNUITY|SK_ID_CURR|
+-----------+----------+
|21181.5    |215354    |
+-----------+----------+



In [43]:
sdf_bureau.select('CREDIT_CURRENCY').distinct().show(10, False)

+---------------+
|CREDIT_CURRENCY|
+---------------+
|currency 2     |
|currency 1     |
|currency 4     |
|currency 3     |
+---------------+



In [25]:
sdf_train.filter(F.col('SK_ID_CURR') == 215354).show(10, False)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-------------------+------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-----------------+------------------+-----------------+------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----

In [59]:
for i in sdf_train.columns:
    for j in sdf_bureau.columns:
        if (i == j):
            print (i)

SK_ID_CURR
AMT_ANNUITY


In [55]:
type(sdf_train.columns)

list

In [13]:
sdf_train.count()

307511

In [11]:
sdf_bureau.count()

1716428

In [24]:
len(sdf_train.columns)

122